# PRIMERA FUENTE DE DATOS: World Bank

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de World Bank para la anemia infantil
base_url = "http://api.worldbank.org/v2/country/ALL/indicator/SH.ANM.CHLD.ZS"

# Definir los parámetros para la solicitud
params = {
    "format": "json",  # Especificar el formato JSON
}

# Lista para almacenar los datos
all_data = []

# Hacer la solicitud inicial a la API
response = requests.get(base_url, params=params)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta a JSON
    
    # Extraer los datos necesarios
    if len(data) > 1:
        all_data = data[1]  # Almacenar los registros de los indicadores
    else:
        print("No se encontraron datos en la respuesta.")
else:
    print("Error al obtener los datos:", response.status_code)

# Convertir los datos extraídos a un DataFrame de Pandas
df_worldbank = pd.json_normalize(all_data)

# Guardar el DataFrame como archivo CSV
output_file = "world_bank_anemia.csv"
df_worldbank.to_csv(output_file, index=False, encoding="utf-8")
print(f"Archivo CSV creado exitosamente: {output_file}")

# SEGUNDA FUENTE: Global Health Observatory

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de nutrición de anemia infantil
base_url = "https://ghoapi.azureedge.net/api/NUTRITION_ANAEMIA_CHILDREN_NUM"

# Definir los parámetros para paginación (si la API la soporta)
params = {
    "$top": 1000,  # Número de registros por página
    "$skip": 0,    # Desplazamiento para la paginación
}

# Lista para almacenar todos los datos
all_data = []

# Hacer varias solicitudes hasta obtener todos los datos
while True:
    response = requests.get(base_url, params=params)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        data = response.json()  # Convertir la respuesta a JSON
        all_data.extend(data["value"])  # Agregar los datos a la lista
        
        # Si no hay más datos (menor cantidad de datos que "$top"), se detiene el ciclo
        if len(data["value"]) < params["$top"]:
            break
        
        # Aumentar el desplazamiento para la siguiente página
        params["$skip"] += params["$top"]
    else:
        print("Error al obtener los datos:", response.status_code)
        break

# Convertir los datos a un DataFrame de Pandas
df_gho = pd.json_normalize(all_data)

# Guardar el DataFrame como archivo CSV
output_file = "gho_anemia.csv"
df_gho.to_csv(output_file, index=False, encoding="utf-8")
print(f"Archivo CSV creado exitosamente: {output_file}")

# TERCERA FUENTE: DHS Program

### ENCUESTA 1: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON CUALQUIER TIPO DE ANEMIA

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de DHS (Nutrición)
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_ANY"

# Definir los parámetros para la solicitud (si soporta paginación)
params = {
    "perpage": 1000,  # Número de registros por página
    "page": 1         # Página inicial
}

# Lista para almacenar todos los datos
all_data = []

# Hacer varias solicitudes hasta obtener todos los datos
while True:
    # Realizar la solicitud a la API
    response = requests.get(base_url, params=params)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta a JSON
        data = response.json()
        
        # Agregar los datos obtenidos a la lista
        all_data.extend(data.get("Data", []))
        
        # Verificar si se ha llegado al final de los resultados
        if len(data.get("Data", [])) < params["perpage"]:
            break
        
        # Incrementar la página para la siguiente solicitud
        params["page"] += 1
    else:
        print("Error al obtener los datos:", response.status_code)
        break

# Convertir los datos a un DataFrame de Pandas
df_any = pd.json_normalize(all_data)

# Guardar el DataFrame como archivo CSV
output_file = "df_any_anemia.csv"
df_any.to_csv(output_file, index=False, encoding="utf-8")
print(f"Archivo CSV creado exitosamente: {output_file}")